In [1]:
import os

# Ensure the directory exists
os.makedirs('Model', exist_ok=True)
os.makedirs('Excel', exist_ok=True)
os.makedirs('Gambar', exist_ok=True)
os.makedirs('Gambar/Heatmap', exist_ok=True)
os.makedirs('Gambar/Best Worst', exist_ok=True)

In [2]:
import pandas as pd

# Load the data from the provided file paths
input_data_path = 'Data/Data Testing Real (Input).csv'
output_data_path = 'Data/Data Testing Real (Output).csv'

# Read the CSV files without headers
input_data = pd.read_csv(input_data_path)
output_data = pd.read_csv(output_data_path)

# Split input data into two arrays: first column and the rest of the columns
data_coor = input_data.iloc[:, 0].values  # First column
data_x = input_data.iloc[:, 1:].values  # Remaining columns
data_y = output_data.values # Output data

In [3]:
from joblib import load

# Assign the Column Name
columns_name = ['Window', 'AC', 'Fan', 'Solar Radiation', 'Outdoor Temperature', 'Wind Velocity', 'Wind Direction', 'Outdoor Relative Humidity']

# Load the Scaler
scaler_X = load('Model/Scaler_X.joblib')
scaler_Y = load('Model/Scaler_Y.joblib')

# Transform the data
data_x = pd.DataFrame(data_x)
data_x.columns = columns_name
data_x = scaler_X.transform(data_x)


In [4]:
import torch
import torch.optim as optim
import torch.nn as nn

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert data to tensor if needed and move to the correct device
Tensor_X = torch.tensor(data_x, dtype=torch.float32).to(device)

# Define your model architecture
class ANNModel(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout_rate=0.3):
        super(ANNModel, self).__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))  # Dropout to prevent overfitting
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, output_dim))  # Output layer (no activation for regression)
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Load the best hyperparameters
best_hyperparameters = torch.load('Model/Best_Hyperparameters.pth')

# Extract each parameter
best_hidden_layers = best_hyperparameters['hidden_layers']
best_hidden_units = [best_hyperparameters[f'n_units_l{i}'] for i in range(best_hidden_layers)]
best_dropout_rate = best_hyperparameters['dropout_rate']
best_lr = best_hyperparameters['lr']
best_optimizer_name = best_hyperparameters['optimizer']

# Re-initialize the model with the loaded best hyperparameters
model = ANNModel(
    input_dim=8,
    hidden_dims=best_hidden_units,
    output_dim=462,
    dropout_rate=best_dropout_rate
).to(device)  # Ensure 'device' is defined (e.g., device = torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Set up the optimizer using the loaded hyperparameter
best_optimizer = getattr(optim, best_optimizer_name)(model.parameters(), lr=best_lr)

# Define the loss criterion
criterion = nn.MSELoss()

# Load the best model weights
model.load_state_dict(torch.load('Model/ANN_Model.pt'))
model.eval()  # Set the model to evaluation mode

# Predict with the model
with torch.no_grad():  # No gradients needed for inference
    predictions = model(Tensor_X)

# Convert predictions to a numpy array or DataFrame as needed
y_pred = predictions.cpu().numpy()

C:\Users\Athal\AppData\Local\Temp\ipykernel_12436\1626799341.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_hyperparameters = torch.load('Model/Best_Hyperparamete

In [5]:
import numpy as np

# Inverse transform the scaled data
y_pred = scaler_Y.inverse_transform(y_pred)

# Initialize a list to store each reshaped array
reshaped_data = []

# Iterate through each row and reshape it into (154, 3)
for row in y_pred:
    reshaped_row = row.reshape(154, 3)
    reshaped_data.append(reshaped_row)

# Convert the list to a NumPy array if needed
reshaped_data = np.array(reshaped_data)

In [7]:
# Dictionary coordinate ID to coordinate
case_dict = {
    1: 123,
    2: 118,
    3: 114,
    4: 39,
    5: 34,
    6: 30,
}

result_data = []

# Get the point coordinate values
for i, coor in enumerate(data_coor):
    # Get the reshaped data for the current point
    data_point = reshaped_data[i, case_dict[coor]]
    result_data.append(data_point)

result_array = np.array([arr.reshape(3) for arr in result_data])

In [10]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Calculate metrics definition
def calculate_metrics(true, pred):
    r2 = r2_score(true, pred, multioutput='raw_values')
    mse = mean_squared_error(true, pred)
    mae = mean_absolute_error(true, pred)
    return r2, mse, mae

# Calculate score
r2_train, mse_train, mae_train = calculate_metrics(data_y, result_array)


# Output the results
print("\n===== Model Performance =====")
print(f"  R2 Score: {r2_train}")
print(f"  MSE: {mse_train:.4f}")
print(f"  MAE: {mae_train:.4f}")



===== Model Performance =====
  R2 Score: [-0.19682902 -0.85306168 -1.59268381]
  MSE: 48.1364
  MAE: 3.6711
